# 20 - Flat NER - Experiment #1

Run Flat NER approach on ground-truth dataset

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

2


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [4]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_0')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m0_flat_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m0_flat_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_0


In [5]:
# CONTROLS
RUN_CAMEMBERT = False            # Set to false to skip training Camembert
RUN_CAMEMBERT_PRETRAINED = True  # Set to false to skip training Camembert pretrained

# Number of times a model will be trained & evaluated on each of the 8 trainsets.
N_RUNS = 5

In [6]:
assert RUN_CAMEMBERT != RUN_CAMEMBERT_PRETRAINED

In [7]:
if RUN_CAMEMBERT:
    MODEL = "Jean-Baptiste/camembert-ner"
    MODEL_NAME = 'camembert_ner'
    FOLDER = f"21-flat-ner-ref-{MODEL_NAME}"
    
if RUN_CAMEMBERT_PRETRAINED:
    MODEL = "HueyNemud/das22-10-camembert_pretrained"
    MODEL_NAME = 'pretrained_camembert_ner'
    FOLDER = f"22-flat-ner-ref-{MODEL_NAME}"

## Constants

In [8]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"01-experiment_1_prepared_ref_dataset_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "20-experiment_1_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_0/01-experiment_1_prepared_ref_dataset_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_0/20-experiment_1_metrics'))

## 20 - Train and eval on reference dataset

In [9]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100,
    "save_total_limit": 1,
}

In [10]:
from config import logger
from datasets import load_from_disk
import json
from camembert_util import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

2023-01-19 14:21:13.602457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 14:21:16.247886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-19 14:21:16.247997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-19 14:21:16.248011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## 21 - CamemBERT - train & eval on reference dataset

In [11]:
import time
import datetime

if RUN_CAMEMBERT:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    print(MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH)
    
    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning CamemBERT model")

Skipped finetuning model


## 22 - CamemBERT pretrained - train & eval

In [12]:
import time
import datetime

if RUN_CAMEMBERT_PRETRAINED:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH
    print(MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH)

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning Pretrained CamemBERT model")

/work/stual/res_ICDAR/method_0/20-experiment_1_metrics/22-flat-ner-ref-pretrained_camembert_ner /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner


Some weights of the model checkpoint at HueyNemud/das22-10-camembert_pretrained were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at HueyNemud/das22-10-camembert_pretrained and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this mo

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Per,Act,Loc,Cardinal,Ft,Titre
100,No log,0.204963,0.941950,0.961252,0.951503,0.965367,"{'precision': 0.9297994269340975, 'recall': 0.9558173784977909, 'f1': 0.9426289034132171, 'number': 679}","{'precision': 0.9155963302752294, 'recall': 0.9614643545279383, 'f1': 0.9379699248120301, 'number': 519}","{'precision': 0.9383033419023136, 'recall': 0.9592641261498029, 'f1': 0.9486679662118258, 'number': 761}","{'precision': 0.9854014598540146, 'recall': 0.995575221238938, 'f1': 0.9904622157006603, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.8181818181818182, 'recall': 0.675, 'f1': 0.7397260273972603, 'number': 40}"
200,No log,0.126874,0.949927,0.968331,0.959041,0.971822,"{'precision': 0.9708029197080292, 'recall': 0.979381443298969, 'f1': 0.9750733137829911, 'number': 679}","{'precision': 0.9122486288848263, 'recall': 0.9614643545279383, 'f1': 0.9362101313320825, 'number': 519}","{'precision': 0.9309462915601023, 'recall': 0.9566360052562418, 'f1': 0.9436163318211277, 'number': 761}","{'precision': 0.9911894273127754, 'recall': 0.995575221238938, 'f1': 0.9933774834437086, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.7804878048780488, 'recall': 0.8, 'f1': 0.7901234567901235, 'number': 40}"
300,No log,0.091654,0.968136,0.973547,0.970834,0.980511,"{'precision': 0.9867256637168141, 'recall': 0.9852724594992637, 'f1': 0.9859985261606484, 'number': 679}","{'precision': 0.945179584120983, 'recall': 0.9633911368015414, 'f1': 0.9541984732824428, 'number': 519}","{'precision': 0.9584415584415584, 'recall': 0.9697766097240473, 'f1': 0.9640757674722404, 'number': 761}","{'precision': 0.9926470588235294, 'recall': 0.995575221238938, 'f1': 0.9941089837997055, 'number': 678}","{'precision': 0.3333333333333333, 'recall': 0.14285714285714285, 'f1': 0.2, 'number': 7}","{'precision': 0.7692307692307693, 'recall': 0.75, 'f1': 0.7594936708860761, 'number': 40}"
400,No log,0.080330,0.966089,0.976528,0.971280,0.979394,"{'precision': 0.9809941520467836, 'recall': 0.9882179675994109, 'f1': 0.9845928099779898, 'number': 679}","{'precision': 0.9275092936802974, 'recall': 0.9614643545279383, 'f1': 0.9441816461684011, 'number': 519}","{'precision': 0.9647058823529412, 'recall': 0.9697766097240473, 'f1': 0.9672346002621232, 'number': 761}","{'precision': 0.9911894273127754, 'recall': 0.995575221238938, 'f1': 0.9933774834437086, 'number': 678}","{'precision': 0.75, 'recall': 0.42857142857142855, 'f1': 0.5454545454545454, 'number': 7}","{'precision': 0.8536585365853658, 'recall': 0.875, 'f1': 0.8641975308641976, 'number': 40}"
500,0.215700,0.082930,0.964484,0.971311,0.967886,0.979022,"{'precision': 0.9750733137829912, 'recall': 0.979381443298969, 'f1': 0.9772226304188096, 'number': 679}","{'precision': 0.9357277882797732, 'recall': 0.953757225433526, 'f1': 0.9446564885496184, 'number': 519}","{'precision': 0.9572538860103627, 'recall': 0.9710906701708278, 'f1': 0.964122635355512, 'number': 761}","{'precision': 0.9911894273127754, 'recall': 0.995575221238938, 'f1': 0.9933774834437086, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.8461538461538461, 'recall': 0.825, 'f1': 0.8354430379746836, 'number': 40}"
600,0.215700,0.080599,0.971503,0.978018,0.974749,0.981380,"{'precision': 0.9823788546255506, 'recall': 0.9852724594992637, 'f1': 0.9838235294117648, 'number': 679}","{'precision': 0.947069943289225, 'recall': 0.9653179190751445, 'f1': 0.9561068702290076, 'number': 519}","{'precision': 0.9660574412532638, 'recall': 0.9724047306176085, 'f1': 0.9692206941715783, 'number': 761}","{'precision': 0.9926470588235294, 'recall': 0.995575221238938, 'f1': 0.9941089837997055, 'number': 678}","{'precision': 0.75, 'recall': 0.42857142857142855, 'f1': 0.5454545454545454, 'number': 7}","{'precision': 0.8809523809523809, 'recall': 0.925, 'f1': 0.9024390243902439, 'number': 40}"


The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/stage_DAS/m0_flat_ner/camembert_util.py:101: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to lo

Trainer is attempting to log a value of "{'precision': 0.7692307692307693, 'recall': 0.75, 'f1': 0.7594936708860761, 'number': 40}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-300
Configuration saved in /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-300/config.json
Model weights saved in /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-300/pytorch_model.bin
tokenizer config file saved in /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-300/tokenizer_config.json
Special tokens file saved in /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-300/special_tokens_map.json
Deleting older checkpoint [/work/stual/res_ICDAR/me

...

Trainer is attempting to log a value of "{'precision': 0.9911894273127754, 'recall': 0.995575221238938, 'f1': 0.9933774834437086, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6666666666666666, 'recall': 0.2857142857142857, 'f1': 0.4, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.925, 'recall': 0.925, 'f1': 0.925, 'number': 40}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /work/stual/res_ICDAR/method_0/tmp/22-flat-ner-ref-pretrained_camembert_ner/checkpoint-900
Configuration saved in /work/stual/res_I

Trainer is attempting to log a value of "{'precision': 0.9660130718954248, 'recall': 0.9710906701708278, 'f1': 0.9685452162516384, 'number': 761}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9911894273127754, 'recall': 0.995575221238938, 'f1': 0.9933774834437086, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.5, 'recall': 0.42857142857142855, 'f1': 0.4615384615384615, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9, 'recall': 0.9, 'f1': 0.9, 'number': 40}" of type <class 'di

Trainer is attempting to log a value of "{'precision': 0.950381679389313, 'recall': 0.9595375722543352, 'f1': 0.9549376797698945, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9572538860103627, 'recall': 0.9710906701708278, 'f1': 0.964122635355512, 'number': 761}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9926470588235294, 'recall': 0.995575221238938, 'f1': 0.9941089837997055, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6, 'recall': 0.42857142857142855, 'f1': 0.5, 'n

Trainer is attempting to log a value of "{'precision': 0.9864146485528648, 'recall': 0.9905100830367735, 'f1': 0.9884581237052383, 'number': 1686}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9066079295154185, 'recall': 0.9405850091407678, 'f1': 0.9232839838492597, 'number': 1094}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9661487236403996, 'recall': 0.974258533855624, 'f1': 0.9701866815268878, 'number': 1787}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9965733866362079, 'recall': 0.9965733866362079,

Run-time is equal to 0:04:14.299347
